# Sensitive analysis

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import pickle

from util import save_model, summarize_results, calculate_95_ci

## Loading data

In [3]:
df_deaths = pd.read_csv('data/output/df_mortality.csv', index_col=0)
df_labeled_cluster = pd.read_csv('data/output/df_labeled_cluster.csv', index_col=0)
df_deaths['cluster_label'] = df_labeled_cluster['cluster_label']

## Variable Sensitivity Analysis

In [4]:
df_difference_r2 = pd.DataFrame()
df_difference_coefficients = pd.DataFrame()

list_periods = ['2020_1','2020', '2021', '2022', '2020_2022']

for model_id in [2,3]:   
    for i in range(5):
        period = list_periods[i]
        print('\n*** Period: ', period)
 
        print('\*** Model',model_id)
        print('===>Full model:')
        with open('models/model_'+str(model_id)+'_'+period+'.pkl', 'rb') as file:
            model_reference = pickle.load(file)
        # summarize_results(model_reference)
        model_reference_r2 = model_reference.pseudo_rsquared()
                
        for removed_variable in model_reference.params[1:].index:
            print('\n*** Removed variable: ', removed_variable)        
            with open('models/sensitivity_analysis/variable/model_' + str(model_id) +'_' + period +'_' + removed_variable + '.pkl', 'rb') as file:
                model_variable = pickle.load(file)        
            # summarize_results(model_variable)
            model_variable_r2 = model_variable.pseudo_rsquared()
            
            r2_difference = model_variable_r2 - model_reference_r2
            r2_row = {'model_id':model_id, 'period': period, 'variable':removed_variable, 'r2_difference':r2_difference}
            new_row = pd.DataFrame([r2_row])
            df_difference_r2 = pd.concat([df_difference_r2, new_row], ignore_index=True)
            
            for variable in model_variable.params.index:
                coefficient_difference = model_variable.params[variable] - model_reference.params[variable]
                coefficient_relative_difference = coefficient_difference / model_reference.params[variable]
                coefficient_row = {'model_id': model_id, 'period': period, 'removed_variable': removed_variable, 'variable': variable, 'coefficient_difference': coefficient_difference, 'coefficient_relative_difference': coefficient_relative_difference}
                new_row = pd.DataFrame([coefficient_row])
                df_difference_coefficients = pd.concat([df_difference_coefficients, new_row], ignore_index=True)


*** Period:  2020_1
\*** Model 2
===>Full model:

*** Removed variable:  cluster_label_Urbanized with informal settlements

*** Removed variable:  cluster_label_Semi-urbanized

*** Removed variable:  cluster_label_Rural with high human development

*** Removed variable:  cluster_label_Rural with low human development

*** Removed variable:  percentage_population_age_range_60_more

*** Removed variable:  percentage_urban_population

*** Removed variable:  demographic_density

*** Removed variable:  percentage_male_population

*** Removed variable:  percentage_indigenous_population

*** Removed variable:  per_capita_income

*** Removed variable:  gini

*** Removed variable:  percentage_estimated_households_in_informal_settlements

*** Removed variable:  demographic_density_in_informal_settlements

*** Removed variable:  percentage_hospitalizations_diseases_inadequate_sanitation

*** Removed variable:  percentage_self_employed_workers

*** Removed variable:  unemployment_rate

*** Remove

In [5]:
df_difference_r2['r2_difference_absolute'] = df_difference_r2['r2_difference'].abs()
df_difference_r2.sort_values(['model_id', 'period', 'r2_difference'], ascending=[True, True, True])[['model_id', 'period', 'variable', 'r2_difference']].round(3)

,model_id,period,variable,r2_difference
41,2,2020,delta_first_death_period,-0.242
25,2,2020,percentage_population_age_range_60_more,-0.006
26,2,2020,percentage_urban_population,-0.004
38,2,2020,percentage_workers_services,-0.004
29,2,2020,percentage_indigenous_population,-0.004
...,...,...,...,...
123,3,2022,Semi-urbanized,0.000
124,3,2022,Rural with high human development,0.000
125,3,2022,Urbanized with informal settlements,0.000
126,3,2022,Rural with low human development,0.000


In [6]:
df_difference_r2[df_difference_r2['model_id'] == 2].groupby(['period'])['r2_difference'].describe().round(3)

,count,mean,std,min,25%,50%,75%,max
period,,,,,,,,
2020,21.0,-0.013,0.053,-0.242,-0.003,-0.001,-0.0,-0.0
2020_1,21.0,-0.017,0.067,-0.310,-0.002,-0.001,-0.0,-0.0
2020_2022,21.0,-0.006,0.016,-0.074,-0.005,-0.001,-0.0,-0.0
2021,21.0,-0.012,0.024,-0.101,-0.006,-0.001,-0.0,-0.0
2022,21.0,-0.019,0.060,-0.266,-0.002,-0.001,-0.0,-0.0


In [7]:
df_difference_r2[df_difference_r2['model_id'] == 3].groupby(['period'])['r2_difference'].describe().round(3)

,count,mean,std,min,25%,50%,75%,max
period,,,,,,,,
2020,6.0,-0.049,0.120,-0.295,0.000,0.000,0.000,0.000
2020_1,6.0,-0.063,0.146,-0.361,-0.004,-0.004,-0.004,-0.004
2020_2022,6.0,-0.014,0.035,-0.085,0.000,0.000,0.000,0.000
2021,6.0,-0.013,0.031,-0.076,0.000,0.000,0.000,0.000
2022,6.0,-0.042,0.103,-0.251,0.000,0.000,0.000,0.000


In [8]:
df_difference_coefficients['coefficient_relative_difference_absolute'] = df_difference_coefficients['coefficient_relative_difference'].abs()

In [9]:
df_difference_coefficients.groupby(['model_id','period','removed_variable'])[['coefficient_relative_difference_absolute']].mean().round(3).reset_index().sort_values(by=['model_id','period','coefficient_relative_difference_absolute'], ascending=[True, True, False])

,model_id,period,removed_variable,coefficient_relative_difference_absolute
4,2,2020,delta_first_death_period,1.225
14,2,2020,percentage_population_age_range_60_more,0.495
2,2,2020,cluster_label_Semi-urbanized,0.388
9,2,2020,per_capita_income,0.362
19,2,2020,percentage_workers_services,0.359
...,...,...,...,...
129,3,2022,Rural with high human development,1.255
133,3,2022,Urbanized with informal settlements,0.571
132,3,2022,Urbanized,0.444
131,3,2022,Semi-urbanized,0.371


In [10]:
df_difference_coefficients.groupby(['model_id','period','removed_variable'])[['coefficient_relative_difference_absolute']].median().round(3).reset_index().sort_values(by=['model_id','period','coefficient_relative_difference_absolute'], ascending=[True, True, False])

,model_id,period,removed_variable,coefficient_relative_difference_absolute
4,2,2020,delta_first_death_period,0.651
16,2,2020,percentage_urban_population,0.241
19,2,2020,percentage_workers_services,0.241
14,2,2020,percentage_population_age_range_60_more,0.215
18,2,2020,percentage_workers_industry,0.180
...,...,...,...,...
129,3,2022,Rural with high human development,1.280
133,3,2022,Urbanized with informal settlements,0.423
132,3,2022,Urbanized,0.351
131,3,2022,Semi-urbanized,0.309


## Parameter sensitivity analysis

In [11]:
df_difference_r2 = pd.DataFrame()
df_difference_coefficients = pd.DataFrame()

list_periods = ['2020_1','2020', '2021', '2022', '2020_2022']

# for model_id in [2,3]:
for model_id in [5]:
    for i in range(5):
        period = list_periods[i]
        
        print('\n*** Period: ', period)
        print('\*** Model',model_id)
        print('===>Full model:')
        with open('models/model_'+str(model_id)+'_'+period+'.pkl', 'rb') as file:
            model_reference = pickle.load(file)        
        # summarize_results(model_reference)
        model_reference_r2 = model_reference.pseudo_rsquared()
        
        for sample in range(30): 
            print('\n*** Sample: ', sample)        
            with open('models/sensitivity_analysis/parameter/model_' + str(model_id) +'_' + period +'_sample_' + str(sample) + '.pkl', 'rb') as file:
                model_sample = pickle.load(file)        
            # summarize_results(model_variable)
            model_sample_r2 = model_sample.pseudo_rsquared()
            
            r2_difference = model_sample_r2 - model_reference_r2
            r2_row = {'model_id':model_id, 'period': period, 'sample':sample, 'r2_difference':r2_difference}
            new_row = pd.DataFrame([r2_row])
            df_difference_r2 = pd.concat([df_difference_r2, new_row], ignore_index=True)
            
            for variable in model_sample.params.index:
                coefficient_difference = model_sample.params[variable] - model_reference.params[variable]
                coefficient_relative_difference = coefficient_difference / model_reference.params[variable]
                rate_ratio_difference = np.exp(model_sample.params[variable]) - np.exp(model_reference.params[variable])
                coefficient_row = {'model_id': model_id, 'period': period, 'sample': sample, 'variable': variable, 'coefficient_difference': coefficient_difference, 'coefficient_relative_difference': coefficient_relative_difference, 'rate_ratio_difference': rate_ratio_difference}
                new_row = pd.DataFrame([coefficient_row])
                df_difference_coefficients = pd.concat([df_difference_coefficients, new_row], ignore_index=True)


*** Period:  2020_1
\*** Model 5
===>Full model:

*** Sample:  0

*** Sample:  1

*** Sample:  2

*** Sample:  3

*** Sample:  4

*** Sample:  5

*** Sample:  6

*** Sample:  7

*** Sample:  8

*** Sample:  9

*** Sample:  10

*** Sample:  11

*** Sample:  12

*** Sample:  13

*** Sample:  14

*** Sample:  15

*** Sample:  16

*** Sample:  17

*** Sample:  18

*** Sample:  19

*** Sample:  20

*** Sample:  21

*** Sample:  22

*** Sample:  23

*** Sample:  24

*** Sample:  25

*** Sample:  26

*** Sample:  27

*** Sample:  28

*** Sample:  29

*** Period:  2020
\*** Model 5
===>Full model:

*** Sample:  0

*** Sample:  1

*** Sample:  2

*** Sample:  3

*** Sample:  4

*** Sample:  5

*** Sample:  6

*** Sample:  7

*** Sample:  8

*** Sample:  9

*** Sample:  10

*** Sample:  11

*** Sample:  12

*** Sample:  13

*** Sample:  14

*** Sample:  15

*** Sample:  16

*** Sample:  17

*** Sample:  18

*** Sample:  19

*** Sample:  20

*** Sample:  21

*** Sample:  22

*** Sample:  23

***

In [13]:
df_difference_r2['r2_difference_absolute'] = df_difference_r2['r2_difference'].abs()
df_difference_r2[df_difference_r2['model_id']==5].groupby(['period'])['r2_difference'].describe().round(3)

,count,mean,std,min,25%,50%,75%,max
period,,,,,,,,
2020,30.0,0.005,0.014,-0.018,-0.005,0.005,0.014,0.036
2020_1,30.0,0.005,0.016,-0.017,-0.008,0.005,0.014,0.037
2020_2022,30.0,0.018,0.023,-0.023,-0.002,0.021,0.037,0.058
2021,30.0,0.002,0.014,-0.028,-0.007,0.003,0.010,0.035
2022,30.0,0.000,0.014,-0.025,-0.007,0.003,0.009,0.036


In [14]:
list_period_labels = ['2020 (first half)','2020','2021','2022','2020-2022']

In [15]:
summary_df = df_difference_r2.groupby(['model_id', 'period'])['r2_difference'].apply(calculate_95_ci).apply(pd.Series).reset_index()
summary_df.columns = ['model_id', 'period', 'mean', 'lower_ci', 'upper_ci']

summary_df['mean'] = summary_df['mean'].round(3)
summary_df['lower_ci'] = summary_df['lower_ci'].round(3)
summary_df['upper_ci'] = summary_df['upper_ci'].round(3)

summary_df['CI'] = summary_df.apply(lambda row: f"{row['mean']}\n({row['lower_ci']}, {row['upper_ci']})", axis=1)

final_summary_df = summary_df[['model_id', 'period','CI']].sort_values('period').pivot(index='model_id', columns='period', values='CI')

final_summary_df = final_summary_df.rename(columns={'2020_1': '2020 (first half)', '2020_2022': '2020-2022'})

final_summary_df = final_summary_df[list_period_labels].reset_index()
final_summary_df.to_csv('data/output/df_sensitivity_analysis_bootstrap_r2_difference.csv', index=False)

In [40]:
final_summary_df

period,model_id,2020 (first half),2020,2021,2022,2020-2022
0,2,"0.005\n(-0.001, 0.01)","0.004\n(-0.002, 0.009)","0.004\n(0.001, 0.008)","-0.001\n(-0.006, 0.004)","0.01\n(0.003, 0.018)"
1,3,"-0.002\n(-0.007, 0.002)","-0.003\n(-0.008, 0.002)","0.0\n(-0.006, 0.006)","-0.003\n(-0.009, 0.003)","0.011\n(0.003, 0.019)"


In [16]:
df_difference_coefficients['coefficient_relative_difference_absolute'] = df_difference_coefficients['coefficient_relative_difference'].abs()

In [18]:
df_difference_coefficients.groupby(['model_id','period','variable'])[['coefficient_relative_difference_absolute']].mean().round(3).reset_index().sort_values(by=['model_id','period','coefficient_relative_difference_absolute'], ascending=[True, True, False])['variable'].unique()

array(['demographic_density_in_informal_settlements',
       'expected_years_of_schooling_at_age_18', 'demographic_density',
       'unemployment_rate', 'percentage_workers_commerce',
       'Rural with low human development', 'percentage_male_population',
       'percentage_hospitalizations_diseases_inadequate_sanitation',
       'gini', 'percentage_self_employed_workers', 'Semi-urbanized',
       'Urbanized with informal settlements', 'per_capita_income',
       'percentage_workers_industry', 'Rural with high human development',
       'Urbanized', 'percentage_population_age_range_60_more',
       'percentage_indigenous_population', 'percentage_urban_population',
       'percentage_workers_services',
       'percentage_estimated_households_in_informal_settlements',
       'delta_first_death_period', 'const'], dtype=object)

In [21]:
summary_df = df_difference_coefficients[df_difference_coefficients['model_id']==5].groupby(['period','variable'])['rate_ratio_difference'].apply(calculate_95_ci).apply(pd.Series).reset_index()

summary_df.columns = ['period','variable', 'mean', 'lower_ci', 'upper_ci']

summary_df['mean'] = summary_df['mean'].round(3)
summary_df['lower_ci'] = summary_df['lower_ci'].round(3)
summary_df['upper_ci'] = summary_df['upper_ci'].round(3)

summary_df['CI'] = summary_df.apply(lambda row: f"{row['mean']}\n({row['lower_ci']}, {row['upper_ci']})", axis=1)

final_summary_df = summary_df[['period','variable','CI']].sort_values('period').pivot(index='variable', columns='period', values='CI')

final_summary_df = final_summary_df.rename(columns={'2020_1': '2020 (first half)', '2020_2022': '2020-2022'})

final_summary_df = final_summary_df[list_period_labels].reset_index()

final_summary_df = final_summary_df.set_index('variable')

final_summary_df = final_summary_df.loc[['const',
                                         'Urbanized',
                                         'Urbanized with informal settlements',
                                         'Semi-urbanized',
                                         'Rural with high human development',
                                         'Rural with low human development',
                                         'percentage_population_age_range_60_more',
                                         'percentage_urban_population',
                                         'demographic_density',
                                         'percentage_male_population',
                                         'percentage_indigenous_population',
                                         'per_capita_income',
                                         'gini',
                                         'percentage_estimated_households_in_informal_settlements',
                                         'demographic_density_in_informal_settlements',
                                         'percentage_hospitalizations_diseases_inadequate_sanitation',
                                         'percentage_self_employed_workers',
                                         'unemployment_rate',
                                         'percentage_workers_commerce',
                                         'percentage_workers_services',
                                         'percentage_workers_industry',
                                         'expected_years_of_schooling_at_age_18',
                                         'delta_first_death_period']].copy()

final_summary_df.index = ['Intercept', 'P(CUrbanized|i)', 'P(CUrbanized with informal settlements|i)','P(CSemi-urbanized|i)','P(CRural with high human development|i)','P(CRural with low human development|i)','% population 60+ years','% urban population','Population density (inhabitants/km²)','% male population','% indigenous population','Per capita income (BRL)','Gini coefficient','% informal settlement households', 'Population density in informal settlement (inhabitants/ha)', '% sanitation-related hospitalizations', '% self-employed workers', 'Unemployment rate', '% commerce workers', '% service workers', '% industry workers', 'Expected years of schooling at age 18', 'Days since first death']

final_summary_df.to_csv('data/output/df_sensitivity_analysis_bootstrap_rate_ratio_difference_model_5.csv', index=True)

In [22]:
final_summary_df

period,2020 (first half),2020,2021,2022,2020-2022
Intercept,"-0.0\n(-0.0, 0.0)","-0.0\n(-0.0, 0.0)","-0.0\n(-0.0, 0.0)","0.0\n(-0.0, 0.0)","0.0\n(-0.0, 0.0)"
P(CUrbanized|i),"0.003\n(-0.006, 0.013)","0.001\n(-0.003, 0.005)","-0.001\n(-0.004, 0.002)","0.001\n(-0.003, 0.005)","0.0\n(-0.002, 0.002)"
P(CUrbanized with informal settlements|i),"0.002\n(-0.007, 0.012)","0.002\n(-0.002, 0.005)","-0.001\n(-0.003, 0.001)","0.001\n(-0.003, 0.005)","0.002\n(-0.002, 0.006)"
P(CSemi-urbanized|i),"-0.003\n(-0.011, 0.005)","0.002\n(-0.001, 0.005)","0.0\n(-0.002, 0.003)","-0.002\n(-0.005, 0.002)","0.0\n(-0.002, 0.002)"
P(CRural with high human development|i),"0.001\n(-0.014, 0.015)","-0.005\n(-0.01, 0.001)","0.001\n(-0.002, 0.004)","0.002\n(-0.004, 0.007)","-0.002\n(-0.007, 0.002)"
P(CRural with low human development|i),"-0.001\n(-0.009, 0.006)","-0.001\n(-0.004, 0.003)","0.0\n(-0.003, 0.003)","-0.001\n(-0.005, 0.003)","0.0\n(-0.001, 0.002)"
% population 60+ years,"-0.008\n(-0.018, 0.002)","-0.0\n(-0.005, 0.005)","-0.0\n(-0.004, 0.004)","-0.003\n(-0.01, 0.004)","0.002\n(-0.002, 0.005)"
% urban population,"0.003\n(-0.011, 0.016)","-0.006\n(-0.013, -0.0)","0.001\n(-0.004, 0.005)","0.002\n(-0.005, 0.01)","0.001\n(-0.003, 0.005)"
Population density (inhabitants/km²),"-0.0\n(-0.004, 0.004)","-0.001\n(-0.004, 0.001)","0.001\n(-0.001, 0.003)","-0.002\n(-0.005, 0.001)","-0.001\n(-0.003, 0.002)"
% male population,"0.001\n(-0.006, 0.009)","-0.002\n(-0.006, 0.002)","0.002\n(-0.001, 0.006)","0.0\n(-0.004, 0.005)","-0.0\n(-0.004, 0.003)"


In [100]:
summary_df = df_difference_coefficients[df_difference_coefficients['model_id']==3].groupby(['period','variable'])['rate_ratio_difference'].apply(calculate_95_ci).apply(pd.Series).reset_index()

In [102]:

summary_df.columns = ['period','variable', 'mean', 'lower_ci', 'upper_ci']

summary_df['mean'] = summary_df['mean'].round(3)
summary_df['lower_ci'] = summary_df['lower_ci'].round(3)
summary_df['upper_ci'] = summary_df['upper_ci'].round(3)

summary_df['CI'] = summary_df.apply(lambda row: f"{row['mean']}\n({row['lower_ci']}, {row['upper_ci']})", axis=1)

final_summary_df = summary_df[['period','variable','CI']].sort_values('period').pivot(index='variable', columns='period', values='CI')

final_summary_df = final_summary_df.rename(columns={'2020_1': '2020 (first half)', '2020_2022': '2020-2022'})

final_summary_df = final_summary_df[list_period_labels].reset_index()

final_summary_df = final_summary_df.set_index('variable')

In [103]:
final_summary_df

period,2020 (first half),2020,2021,2022,2020-2022
variable,,,,,
Rural with high human development,"-0.003\n(-0.01, 0.005)","0.003\n(-0.001, 0.007)","0.001\n(-0.002, 0.003)","-0.002\n(-0.006, 0.003)","0.001\n(-0.001, 0.004)"
Rural with low human development,"-0.005\n(-0.009, -0.0)","-0.0\n(-0.002, 0.002)","0.0\n(-0.002, 0.002)","0.003\n(-0.001, 0.006)","0.002\n(-0.0, 0.004)"
Semi-urbanized,"0.002\n(-0.003, 0.007)","-0.001\n(-0.003, 0.001)","0.001\n(-0.001, 0.002)","-0.001\n(-0.003, 0.001)","-0.0\n(-0.001, 0.0)"
Urbanized,"0.003\n(-0.001, 0.007)","-0.001\n(-0.003, 0.001)","-0.001\n(-0.003, 0.001)","0.0\n(-0.003, 0.003)","-0.001\n(-0.002, 0.0)"
Urbanized with informal settlements,"-0.001\n(-0.005, 0.004)","-0.001\n(-0.003, 0.001)","-0.0\n(-0.002, 0.001)","0.0\n(-0.002, 0.003)","-0.001\n(-0.003, 0.001)"
const,"-0.0\n(-0.0, 0.0)","0.0\n(-0.0, 0.0)","0.0\n(-0.0, 0.0)","0.0\n(-0.0, 0.0)","0.0\n(-0.0, 0.0)"
delta_first_death_period,"0.017\n(-0.0, 0.035)","0.003\n(-0.005, 0.012)","0.003\n(-0.002, 0.008)","-0.015\n(-0.03, 0.0)","0.001\n(-0.002, 0.004)"


In [104]:
final_summary_df = final_summary_df.loc[['const',
                                         'Urbanized',
                                         'Urbanized with informal settlements',
                                         'Semi-urbanized',
                                         'Rural with high human development',
                                         'Rural with low human development',
                                         'delta_first_death_period']].copy()

final_summary_df.to_csv('data/output/df_sensitivity_analysis_bootstrap_rate_ratio_difference_model_3.csv', index=True)

## Outlier Sensitivity Analysis

### Mortality rate ratio using 'Urbanized' as the reference group

In [16]:
df_analysis_cluster = pd.DataFrame()
df_analysis_cluster['Sociodemographic cluster'] = ['Urbanized','Urbanized with informal settlements','Semi-urbanized','Rural with high human development','Rural with low human development','Residual degrees of freedom','Deviance','Pearson chi-squared','$R_{CU}^{2}$','AIC']
df_analysis_cluster = df_analysis_cluster.set_index('Sociodemographic cluster')

list_periods = ['2020_1','2020', '2021', '2022', '2020_2022']
list_death_rate_columns = ['Death rate (1/2020)', 'Death rate (2020)', 'Death rate (2021)', 'Death rate (2022)', 'Death rate (accumulated period)']
list_period_labels = ['2020 (first half)','2020','2021','2022','2020-2022']

for i in range(len(list_periods)):
    period = list_periods[i]
    period_label = list_period_labels[i]
    death_rate_column = list_death_rate_columns[i]
    
    mortality_rate = df_deaths.groupby('cluster_label')[death_rate_column].mean()
    ci_data = df_deaths.groupby('cluster_label')[death_rate_column].apply(calculate_95_ci)
    df_analysis_cluster[['Mean', 'Lower CI', 'Upper CI']] = pd.DataFrame(ci_data.tolist(), index=ci_data.index).round(2)
    df_analysis_cluster[period_label+' - Death rate (95% CI)'] = df_analysis_cluster['Mean'].astype(str) + '\n' + '('+df_analysis_cluster['Lower CI'].astype(str) + '-'+df_analysis_cluster['Upper CI'].astype(str) + ')'
    df_analysis_cluster = df_analysis_cluster.drop(columns=['Mean', 'Lower CI', 'Upper CI'])
    
    model = 2
    model_label = str(model)
    model_file = str(model)
    with open('models/sensitivity_analysis/outliers/model_'+model_file+'_'+period+'.pkl', 'rb') as file:
        model = pickle.load(file)

    # Extract the coefficients and standard errors
    params = model.params[1:5]
    conf = model.conf_int()[1:5]
    conf.columns = ['Lower CI', 'Upper CI']
    
    # Calculate the rate ratios and their confidence intervals
    rate_ratios = np.exp(params)
    conf['Lower CI'] = np.exp(conf['Lower CI'])
    conf['Upper CI'] = np.exp(conf['Upper CI'])
    
    # Combine into a single DataFrame
    rate_ratio_df = pd.DataFrame({
        'Rate Ratio': rate_ratios,
        'Lower CI': conf['Lower CI'],
        'Upper CI': conf['Upper CI']
    }) 
    
    rate_ratio_df.index = rate_ratio_df.index.astype(str).str.replace('cluster_label_','')
    
    rate_ratio_df = rate_ratio_df.round(2)
    
    column_rate_ratio = period_label+' - RR (95% CI) - Model '+model_label
    df_analysis_cluster[column_rate_ratio] = rate_ratio_df['Rate Ratio'].astype(str) + '\n' + '('+rate_ratio_df['Lower CI'].astype(str) + '-'+rate_ratio_df['Upper CI'].astype(str) + ')'        
    
    df_analysis_cluster.loc['Residual degrees of freedom', column_rate_ratio] = model.df_resid
    df_analysis_cluster.loc['Deviance', column_rate_ratio] = round(model.deviance,2)
    df_analysis_cluster.loc['Pearson chi-squared', column_rate_ratio] = round(model.pearson_chi2,2)
    df_analysis_cluster.loc['$R_{CU}^{2}$', column_rate_ratio] = round(model.pseudo_rsquared('cs'),2)
    df_analysis_cluster.loc['AIC', column_rate_ratio] = round(model.aic,2)
    
    
    df_analysis_cluster[column_rate_ratio] = df_analysis_cluster[column_rate_ratio].fillna('1 [Reference]')

df_analysis_cluster.to_csv('data/output/df_analysis_cluster_outlier_sensitivity.csv', index=True)

In [17]:
df_analysis_cluster

,2020 (first half) - Death rate (95% CI),2020 (first half) - RR (95% CI) - Model 2,2020 - Death rate (95% CI),2020 - RR (95% CI) - Model 2,2021 - Death rate (95% CI),2021 - RR (95% CI) - Model 2,2022 - Death rate (95% CI),2022 - RR (95% CI) - Model 2,2020-2022 - Death rate (95% CI),2020-2022 - RR (95% CI) - Model 2
Sociodemographic cluster,,,,,,,,,,
Urbanized,9.24\n(8.56-9.92),1 [Reference],68.86\n(66.9-70.83),1 [Reference],243.29\n(239.23-247.35),1 [Reference],41.82\n(40.58-43.07),1 [Reference],353.97\n(348.7-359.25),1 [Reference]
Urbanized with informal settlements,47.19\n(42.74-51.64),0.95\n(0.83-1.08),115.8\n(110.31-121.3),0.99\n(0.92-1.07),192.68\n(184.22-201.14),0.77\n(0.73-0.82),27.89\n(26.19-29.6),0.93\n(0.87-0.99),336.37\n(323.17-349.58),0.83\n(0.79-0.88)
Semi-urbanized,17.88\n(16.83-18.93),1.4\n(1.24-1.57),63.41\n(61.57-65.26),1.13\n(1.07-1.2),123.63\n(120.71-126.54),0.74\n(0.71-0.77),22.93\n(22.13-23.74),0.87\n(0.83-0.92),209.97\n(206.09-213.85),0.81\n(0.78-0.84)
Rural with high human development,4.58\n(3.72-5.43),1.12\n(0.91-1.38),52.26\n(48.76-55.77),1.21\n(1.13-1.31),193.64\n(186.67-200.62),0.97\n(0.93-1.02),37.97\n(35.51-40.43),1.03\n(0.96-1.11),283.88\n(275.03-292.72),1.02\n(0.98-1.06)
Rural with low human development,30.19\n(27.0-33.39),1.42\n(1.17-1.71),64.4\n(59.79-69.01),1.0\n(0.91-1.11),83.94\n(78.38-89.51),0.56\n(0.52-0.61),11.35\n(10.2-12.5),0.72\n(0.64-0.81),159.69\n(150.64-168.75),0.64\n(0.6-0.68)
Residual degrees of freedom,nan\n(nan-nan),5216,nan\n(nan-nan),5242,nan\n(nan-nan),5216,nan\n(nan-nan),5248,nan\n(nan-nan),5236
Deviance,nan\n(nan-nan),3902.69,nan\n(nan-nan),5112.45,nan\n(nan-nan),5271.27,nan\n(nan-nan),4088.39,nan\n(nan-nan),5114.88
Pearson chi-squared,nan\n(nan-nan),4027.12,nan\n(nan-nan),4811.32,nan\n(nan-nan),5059.39,nan\n(nan-nan),4221.25,nan\n(nan-nan),4945.86
$R_{CU}^{2}$,nan\n(nan-nan),0.74,nan\n(nan-nan),0.5,nan\n(nan-nan),0.74,nan\n(nan-nan),0.67,nan\n(nan-nan),0.68


### Rate ratio for the cluster probabilities

In [18]:
df_analysis_cluster_probabilities = pd.DataFrame()
df_analysis_cluster_probabilities['Variable'] = ['Urbanized (probability)','Urbanized with informal settlements (probability)','Semi-urbanized (probability)','Rural with high human development (probability)','Rural with low human development (probability)','Days since first death','Residual degrees of freedom','Deviance','Pearson chi-squared','$R_{CU}^{2}$','AIC']
df_analysis_cluster_probabilities = df_analysis_cluster_probabilities.set_index('Variable')

for i in range(len(list_periods)):
    period = list_periods[i]
    period_label = list_period_labels[i]
    
    with open('models/sensitivity_analysis/outliers/model_3_'+period+'.pkl', 'rb') as file:
        model = pickle.load(file)

    # Extract the coefficients and standard errors
    params = model.params[1:]
    conf = model.conf_int()[1:]
    conf.columns = ['Lower CI', 'Upper CI']
    
    # Calculate the rate ratios and their confidence intervals
    rate_ratios = np.exp(params)
    conf['Lower CI'] = np.exp(conf['Lower CI'])
    conf['Upper CI'] = np.exp(conf['Upper CI'])
    
    # Combine into a single DataFrame
    rate_ratio_df = pd.DataFrame({
        'Rate Ratio': rate_ratios,
        'Lower CI': conf['Lower CI'],
        'Upper CI': conf['Upper CI']
    })
    
    rate_ratio_df = rate_ratio_df.round(2)
    
    rate_ratio_df.index = rate_ratio_df.index.astype(str) + ' (probability)'
    index_list = rate_ratio_df.index.tolist()
    index_list[-1] = 'Days since first death'
    rate_ratio_df.index = index_list
    
    df_analysis_cluster_probabilities[period_label] = rate_ratio_df['Rate Ratio'].astype(str) + '\n' + '('+rate_ratio_df['Lower CI'].astype(str) + '-'+rate_ratio_df['Upper CI'].astype(str) + ')'
    
    df_analysis_cluster_probabilities.loc['Residual degrees of freedom', period_label] = model.df_resid
    df_analysis_cluster_probabilities.loc['Deviance', period_label] = round(model.deviance,2)
    df_analysis_cluster_probabilities.loc['Pearson chi-squared', period_label] = round(model.pearson_chi2,2)    
    df_analysis_cluster_probabilities.loc['$R_{CU}^{2}$', period_label] = round(model.pseudo_rsquared('cs'),2)
    df_analysis_cluster_probabilities.loc['AIC', period_label] = round(model.aic,2)
    
df_analysis_cluster_probabilities.to_csv('data/output/df_analysis_cluster_probabilities_outlier_sensitivity.csv', index=True)

In [19]:
df_analysis_cluster_probabilities

,2020 (first half),2020,2021,2022,2020-2022
Variable,,,,,
Urbanized (probability),0.8\n(0.78-0.83),1.0\n(0.99-1.01),1.19\n(1.18-1.2),1.11\n(1.1-1.13),1.15\n(1.14-1.15)
Urbanized with informal settlements (probability),1.03\n(1.01-1.05),1.03\n(1.02-1.04),1.0\n(1.0-1.01),0.94\n(0.93-0.95),1.02\n(1.01-1.02)
Semi-urbanized (probability),1.18\n(1.16-1.21),0.99\n(0.98-1.0),0.89\n(0.88-0.89),0.91\n(0.9-0.92),0.91\n(0.9-0.91)
Rural with high human development (probability),0.93\n(0.88-0.98),1.04\n(1.02-1.06),1.06\n(1.05-1.07),1.21\n(1.19-1.23),1.06\n(1.05-1.07)
Rural with low human development (probability),1.15\n(1.13-1.18),0.94\n(0.93-0.95),0.85\n(0.84-0.85),0.81\n(0.79-0.82),0.86\n(0.85-0.87)
Days since first death,2.47\n(2.39-2.55),1.69\n(1.65-1.73),1.24\n(1.22-1.27),2.27\n(2.17-2.37),1.2\n(1.18-1.21)
Residual degrees of freedom,5232,5258,5232,5264,5252
Deviance,3899.91,5244.85,5364.65,4292.77,5201.02
Pearson chi-squared,4076.5,5027.84,5169.82,4643.07,5067.34


### Analysis of the sociodemographic variables

In [20]:
df_analysis_variables = pd.DataFrame()

for i in range(len(list_periods)):
    period = list_periods[i]
    period_label = list_period_labels[i]
    
    with open('models/sensitivity_analysis/outliers/model_2_'+period+'.pkl', 'rb') as file:
        model = pickle.load(file)

    # Extract the coefficients and standard errors
    params = model.params[5:]
    conf = model.conf_int()[5:]
    conf.columns = ['Lower CI', 'Upper CI']
    
    # Calculate the rate ratios and their confidence intervals
    rate_ratios = np.exp(params)
    conf['Lower CI'] = np.exp(conf['Lower CI'])
    conf['Upper CI'] = np.exp(conf['Upper CI'])
    
    # Combine into a single DataFrame
    rate_ratio_df = pd.DataFrame({
        'Rate Ratio': rate_ratios,
        'Lower CI': conf['Lower CI'],
        'Upper CI': conf['Upper CI']
    })
    
    rate_ratio_df = rate_ratio_df.round(2)
    
    df_analysis_variables[period_label] = rate_ratio_df['Rate Ratio'].astype(str) + '\n' + '('+rate_ratio_df['Lower CI'].astype(str) + '-'+rate_ratio_df['Upper CI'].astype(str) + ')'
    
df_analysis_variables['Variable'] = ['% population ≥ 60 years','% urban population','Population density','% males','% indigenous peoples','Per capita income','Gini coefficient','% informal settlement households', 'Population density in informal settlement', '% sanitation-related hospitalizations', '% self-employed workers', 'Unemployment rate', '% commerce workers', '% service workers', '% industry workers', 'Projected educational attainment at age 18','Days since first death']

df_analysis_variables = df_analysis_variables.set_index('Variable')  

df_analysis_variables.to_csv('data/output/df_analysis_variables_sensitivity_analysis.csv', index=True)

In [21]:
df_analysis_variables

,2020 (first half),2020,2021,2022,2020-2022
Variable,,,,,
% population ≥ 60 years,0.92\n(0.88-0.97),1.09\n(1.07-1.12),1.1\n(1.08-1.11),1.34\n(1.31-1.37),1.11\n(1.09-1.12)
% urban population,1.09\n(1.03-1.16),1.11\n(1.08-1.14),1.14\n(1.12-1.16),1.09\n(1.06-1.13),1.15\n(1.13-1.17)
Population density,1.04\n(1.01-1.06),1.01\n(0.99-1.02),1.0\n(0.98-1.01),0.98\n(0.97-0.99),1.0\n(0.99-1.01)
% males,1.0\n(0.96-1.04),1.02\n(1.0-1.04),1.07\n(1.05-1.08),1.08\n(1.06-1.1),1.05\n(1.03-1.06)
% indigenous peoples,1.02\n(0.99-1.04),1.04\n(1.03-1.06),1.03\n(1.02-1.05),1.03\n(1.0-1.05),1.04\n(1.03-1.06)
Per capita income,0.66\n(0.63-0.7),0.92\n(0.89-0.94),1.0\n(0.98-1.02),1.03\n(1.01-1.06),0.99\n(0.97-1.0)
Gini coefficient,1.05\n(1.01-1.1),0.97\n(0.95-1.0),1.0\n(0.98-1.01),0.96\n(0.94-0.99),0.99\n(0.98-1.01)
% informal settlement households,1.09\n(1.07-1.12),1.05\n(1.03-1.06),1.0\n(0.99-1.01),0.99\n(0.97-1.0),1.01\n(1.0-1.02)
Population density in informal settlement,1.01\n(0.98-1.03),1.0\n(0.98-1.01),1.02\n(1.0-1.03),1.0\n(0.99-1.01),1.01\n(1.0-1.02)


### Model 5

In [23]:
df_analysis_models_5 = pd.DataFrame()
df_analysis_models_5['Variables'] = ['const', 'Urbanized', 'Urbanized with informal settlements', 'Semi-urbanized', 'Rural with high human development', 'Rural with low human development', 'percentage_population_age_range_60_more',
       'percentage_urban_population', 'demographic_density',
       'percentage_male_population', 'percentage_indigenous_population',
       'per_capita_income', 'gini',
       'percentage_estimated_households_in_informal_settlements',
       'demographic_density_in_informal_settlements',
       'percentage_hospitalizations_diseases_inadequate_sanitation',
       'percentage_self_employed_workers', 'unemployment_rate',
       'percentage_workers_commerce', 'percentage_workers_services',
       'percentage_workers_industry', 'expected_years_of_schooling_at_age_18',
       'delta_first_death_period','Residual degrees of freedom','Deviance','Pearson chi-squared','$R_{CU}^{2}$','AIC']
df_analysis_models_5 = df_analysis_models_5.set_index('Variables')

list_periods = ['2020_1','2020', '2021', '2022', '2020_2022']
list_death_rate_columns = ['Death rate (1/2020)', 'Death rate (2020)', 'Death rate (2021)', 'Death rate (2022)', 'Death rate (accumulated period)']
list_period_labels = ['2020 (first half)','2020','2021','2022','2020-2022']

for i in range(len(list_periods)):
    period = list_periods[i]
    period_label = list_period_labels[i]
    death_rate_column = list_death_rate_columns[i]
    
    for model in [5]:
        model_label = str(model)
        model_file = str(model)
        
        with open('models/sensitivity_analysis/outliers/model_'+model_file+'_'+period+'.pkl', 'rb') as file:
            model = pickle.load(file)        

        # Extract the coefficients and standard errors
        params = model.params[0:]
        conf = model.conf_int()[0:]
        conf.columns = ['Lower CI', 'Upper CI']
        
        # Calculate the rate ratios and their confidence intervals
        rate_ratios = np.exp(params)
        conf['Lower CI'] = np.exp(conf['Lower CI'])
        conf['Upper CI'] = np.exp(conf['Upper CI'])
        
        # Combine into a single DataFrame
        rate_ratio_df = pd.DataFrame({
            'Rate Ratio': rate_ratios,
            'Lower CI': conf['Lower CI'],
            'Upper CI': conf['Upper CI']
        }) 
        
        rate_ratio_df = rate_ratio_df.round(2)
        
        column_rate_ratio = period_label+' - RR (95% CI) - Model '+model_label
        df_analysis_models_5[column_rate_ratio] = rate_ratio_df['Rate Ratio'].astype(str) + '\n' + '(' + rate_ratio_df['Lower CI'].astype(str) + '-' + rate_ratio_df['Upper CI'].astype(str) + ')'        
        
        df_analysis_models_5.loc['Residual degrees of freedom', column_rate_ratio] = model.df_resid
        df_analysis_models_5.loc['Deviance', column_rate_ratio] = round(model.deviance, 2)
        df_analysis_models_5.loc['Pearson chi-squared', column_rate_ratio] = round(model.pearson_chi2, 2)
        df_analysis_models_5.loc['$R_{CU}^{2}$', column_rate_ratio] = round(model.pseudo_rsquared('cs'), 2)
        df_analysis_models_5.loc['AIC', column_rate_ratio] = round(model.aic, 2)

df_analysis_models_5.index = ['Intercept', 'Urbanized', 'Urbanized with informal settlements', 'Semi-urbanized', 'Rural with high human development', 'Rural with low human development', '% population 60+ years', '% urban population', 'Population density (inhabitants/km²)', '% male population', '% indigenous population', 'Per capita income (BRL)', 'Gini coefficient', '% informal settlement households', 'Population density in informal settlement (inhabitants/ha)', '% sanitation-related hospitalizations', '% self-employed workers', 'Unemployment rate', '% commerce workers', '% service workers', '% industry workers', 'Expected years of schooling at age 18', 'Days since first death', 'Residual degrees of freedom', 'Deviance', 'Pearson chi-squared', '$R_{CU}^{2}$', 'AIC']

df_analysis_models_5.to_csv('data/output/df_analysis_model_5_sensitivity_analysis.csv', index=True)